In [ ]:
/// desc:		通过医嘱ID获取处方号
/// input:		orditem
/// output:		prescNo
/// w ##class(web.DHCDocMain).GetPrescNoByOrdItem("182||5")
ClassMethod GetPrescNoByOrdItem(OrdItem)
{
	q:OrdItem="" ""
	s PrescNo = $p($g(^OEORD(+OrdItem,"I",$p($g(OrdItem),"||",2),1)),"^",14)
	q PrescNo
}

In [ ]:
/// 获取医嘱项代码
/// w ##class(web.DHCDocMain).GetArcimCode("1||1")
ClassMethod GetArcimCode(OrdItem)
{
	q:OrdItem="" ""
	s Arcim = $p($g(^OEORD(+OrdItem,"I",$p(OrdItem,"||",2),1)),"^",2)
	s code = $p($g(^ARCIM(+Arcim,1,1)),"^",1)
	q code
}

In [ ]:
/// 获取检验号
/// w ##class(web.DHCDocMain).GetLabNoByOrdItem("1244||5")
ClassMethod GetLabNoByOrdItem(OrdItem)
{
	q:OrdItem="" ""
	s PrescNo = $p($g(^OEORD(+OrdItem,"I",$p($g(OrdItem),"||",2),3)),"^",20)
	q PrescNo
}


In [ ]:
/// 获取检验主遗嘱
/// w ##class(web.DHCDocMain).GetLabMainOrd("1||1")
ClassMethod GetLabMainOrd(OrdItem)
{
	q:OrdItem="" ""
	s LabNoStr = $p($g(^OEORD(+OrdItem,"I",$p(OrdItem,"||",2),"DHC")),"^",22)
	s LabNo = $p($g(LabNoStr),$c(1))
	s MainOrd = ""
	&sql(select OEORI_RowId into:MainOrd from SQLUser.oe_ordItem where OEORI_LabEpisodeNo = :LabNo)
	q MainOrd
}

In [ ]:
s OrderType=##class(DHCDoc.Order.Common).GetOrdItemType(OrderItemRowId)

In [ ]:
/// Creator: Lid
/// CreatDate: 2017-08-30
/// Description: 提供给其他产品组调用的取医嘱或执行记录的计费状态
/// Input: oeitm:医嘱RowID或执行记录RowID，注：门诊要传OE_OrdItem.RowId，住院要传OE_OrdExec.RowId
/// Return: 账单状态^账单状态描述
/// 		TB^未计费(未收费)，B^已计费(未收费)，P^已收费，I^需退费，R^已退费，-1^入参错误，-10^程序异常
/// Debug: w ##class(web.DHCBillInterface).IGetOrdItmBilled("93||3")

In [ ]:
/// 医嘱 查询
d ##class(web.DHCOEOrdItem).FindOrderItemsNew

In [ ]:
w ##class(web.DHCDocOrderCommon).GetLabSpec("3950||1",2)
/// 医嘱项目 检验标本

In [ ]:
/// 医嘱项目 医院
s:HospitalId="" HospitalId = $g(^ARCIM(+ArcimRowid,1,"HOSP",1))

In [ ]:
/// 接收科室
w ##class(web.DHCDocOrderCommon).GetRecloc(347731,"5068||1","0")

In [ ]:
/// 医嘱 类型代码
s OrderType=##class(DHCDoc.Order.Common).GetOrdItemType(OrdItemID)

In [ ]:
/**
	* @debug w ##class(web.DHCDocMain).CalBaseQty("11134||1","16","4","O")
	* @debug w ##class(web.DHCDocMain).CalBaseQty("11134||1","0.1","66","O")
	* @debug w ##class(web.DHCDocMain).CalBaseQty("1475||1","2","16","O")
*/
ClassMethod CalBaseQty(type, arcim, qty, uom, AdmType)
{
	s ^CalBaseQty(uom) = $lb(arcim, qty, uom, AdmType)
	q:arcim="" ""
	q:qty="" ""
	q:uom="" ""
	s res = ""
	if type="dose" {
		s doseqty = qty
		s drgform=$p($g(^ARCIM(+arcim,+$p(arcim,"||",2),1)),"^",12)
		s res = $$calcqty^DHCOEOrdItem(drgform,uom,doseqty,AdmType)
	}
	if type="pack" {
		s convFac=##class(appcom.OEDispensing).convFac(arcim,uom)
		s res=qty*convFac
	}
	q res
}

ClassMethod GetEqUom(arcim, EpisodeID)
{
	s DrgformRowid=##class(web.DHCDocOrderEntry).GetDrgForm(arcim)
	i DrgformRowid'="" {
		s PHCDRowid=$P(DrgformRowid,"||",1)
		s ChildSub=$P(DrgformRowid,"||",2)

		s HospitalID = ##class(DHCDoc.Common.Hospital).GetAdmHospitalId(EpisodeID)
		s PHUseEqQty=..%GetConfig("PHUseEqQty",HospitalID)
		;草药不取等效单位
		s defaultFindFlag=0,firstEqUomID=""

		s leq=0  f  s leq=$o(^PHCD(PHCDRowid,"DF",ChildSub,"EQ",leq)) q:leq=""  d
		.s eqrec=^PHCD(PHCDRowid,"DF",ChildSub,"EQ",leq)
		.s FormDoseUOMRowid=$p(eqrec,"^"),eqqty=$p(eqrec,"^",2),eqdefaultqty=$p(eqrec,"^",3),defaultFlag=$p(eqrec,"^",4)
		.;s FormDoseQty=eqdefaultqty
		.if eqdefaultqty'="" s FormDoseQty=eqdefaultqty
		.e  d
		..i PHUseEqQty=1 s FormDoseQty=eqqty
		..e  s FormDoseQty=""
		.s FormDoseUOMDesc=##class(web.DHCDocOrderCommon).GetUOMDesc(FormDoseUOMRowid)
		.i (FormDoseQty'="")&(FormDoseQty<1)&(FormDoseQty'=0) s FormDoseQty="0"_FormDoseQty
		.s formDoseQty(FormDoseUOMRowid)=FormDoseQty_$C(1)_FormDoseUOMDesc_$C(1)_FormDoseUOMRowid_$C(1)_defaultFlag_$C(1)_FormDoseQty
		.s:firstEqUomID="" firstEqUomID=FormDoseUOMRowid
		.s:defaultFlag="Y" defaultFindFlag=1
	}
	q $g(firstEqUomID)
}


In [ ]:
/// @debug: w ##class(web.DHCDocMain).CheckPackBaseQty("312031","441||1","3","6")
ClassMethod CheckPackBaseQty(EpisodeID, Arcim, PackQty, PackUom)
{
	/*
	片剂、普通片、分散片、含片、薄膜衣片、咀嚼片、缓释片、控释片、肠溶片、糖衣片、胶囊剂、缓释胶囊剂、软胶囊剂、颗粒剂、散剂、干混悬剂、口服溶液
	*/
	s ^CheckPackBaseQty = $lb(EpisodeID, Arcim, PackQty, PackUom)
	q:EpisodeID="" ""
	q:Arcim="" ""
	q:PackQty="" ""
	q:PackUom="" ""
	s AdmType = $p(^PAADM(EpisodeID),"^",2)
	s HospID=##class(DHCDoc.Common.Hospital).GetAdmHospitalId(EpisodeID)
	q:HospID'=2 ""
	q:AdmType="I" ""
	s list = $lb("片剂","普通片","分散片","含片","薄膜衣片","咀嚼片","缓释片","控释片","肠溶片","糖衣片","胶囊剂","缓释胶囊剂","软胶囊剂","颗粒剂","散剂","干混悬剂","口服溶液")
	s ItemCatDR=$p(^ARCIM(+Arcim,1,1),"^",10)
	s OrderType=$P(^ARC("IC",ItemCatDR),"^",7)
	q:OrderType'="R" ""
	s Phcdf=##class(web.DHCST.Common.DrugInfoCommon).GetPhcdfByArcim(Arcim)
	s drugform=$p(##class(web.DHCST.Common.DrugInfoCommon).GetFormByPhcdf(Phcdf),"^",2)
	if $lf(list, drugform) {
		s convFac=##class(appcom.OEDispensing).convFac(Arcim,PackUom)
		s res=PackQty*convFac
		;整包装单位
		/*
		s PackUOMRowid=$p($g(^ARCIM(+ArcimRowid,1,8)),"^",14)
		s convFac=##class(appcom.OEDispensing).convFac(Arcim,PackUOMRowid)
		if res < convFac {
				
		}
		*/
		s incId=$o(^INCI(0,"ARCIM_DR",+Arcim,0))
		s UOMFac=1
		i incId'="" d
		.//包装单位
		.s bUom=$p(^INCI(incId,1),"^",10)
		.//基本单位
		.s pUom=$p(^INCI(incId,3),"^",6)
		.//门诊发药单位
		.s outPhUomDr=$p(^INCI(incId,1),"^",12)
		.//入库单位（正包装的）
		.s UOMFac= ##class(PHA.FACE.OUT.Com).UOMFac(pUom,bUom) //整包装单位
		.//s UOMFac= ##class(PHA.FACE.OUT.Com).UOMFac(outPhUomDr,bUom) // 和发药单位匹配
		//取医嘱的销售单位  发药单位
		b ;;;;
		if res < $g(UOMFac) {
			q "Y"	
		}
		q ""
	}
	else {
		q ""	
	}
	;剂型
	;基本单位
	;整包装
}

In [ ]:
w ##class(Nur.HISUI.NeedCareOrder).ifUnExcuteExam("223528||79")

In [ ]:
InitPatOrderViewGlobal()
医嘱录入

In [ ]:
w ##class(web.DHCDocOrderCommon).GetLocRecLoc("474","42696||1","","I")
获取接受科室
w ##Class(web.DHCDocOrderCommon).GetRecLocInfo(^TEMP("GetRecLocInfo"))

In [ ]:
s DoseQty=$p($g(^OEORD(OEOrdRowID,"I",OEOrdItemSub,2)),"^",1)		// 单次剂量
		s DoseQtyUomDR=$p($g(^OEORD(OEOrdRowID,"I",OEOrdItemSub,2)),"^",3)		// 单次剂量单位
		i DoseQtyUomDR'="" s DoseQtyUom=$p($g(^CT("UOM",DoseQtyUomDR)),"^",2)
		e  s DoseQtyUom=""
		s PackQty=$p($g(^OEORD(OEOrdRowID,"I",OEOrdItemSub,9)),"^",4)		// 数量 整
		s PackQtyUomDR=$p($g(^OEORD(OEOrdRowID,"I",OEOrdItemSub,"DHC")),"^",13)		// 数量单位

In [ ]:
/// Creator:      郭荣勇
/// CreatDate:    2016.08.11
/// Description:  根据医嘱项取得相应的处方类别
/// Table:        
/// Input:        ArcimRowid:医嘱项指针
/// Return:      处方类别代码:1:西药,2:中成药3:草药4:有频次的处置治疗5:精神二类药6:精神一类药7:抗生素8:毒麻
/// Others: 
web.DHCDocOrderCommon:GetPHPrescTypeByItem

In [ ]:
/// 护理级别 天数
/// w ##class(web.DHCDocMain).GetCareDay(2004834,1)
ClassMethod GetCareDay(EpisodeID, Type)
{
                                                            
	;Set EpisodeID = $o(^PAPERdr(PatientID,"ADM","O",EpisodeID))
	Quit:EpisodeID=""
	Quit:Type=""
	if Type = 1 {
		s Type = "Ⅰ级护理"	
	}
	if Type = 2 {
		s Type = "Ⅱ级护理"	
	}
	if Type = 3 {
		s Type = "Ⅲ级护理"	
	}
	if Type = 0 {
		s Type = "特级护理"	
	}
	s count = 0
	Set ord = 0
	For {
		Set ord = $o(^OEORD(0,"Adm",EpisodeID,ord))
		Quit:ord=""
		;b:ord=2000306
		Set item = 0
		For {
			Set item = $o(^OEORD(ord,"I",item))
            Quit:item=""
            continue:((ord="2037583")&&(item="62"))   ///空医嘱退出
            Set arcim = $p(^OEORD(ord,"I",item,1),"^",2)
            Set desc = $p(^ARCIM(+arcim,1,1),"^",3)
            s desc = $replace(desc,"I","Ⅰ")
            s desc = $replace(desc,"II","Ⅱ")
            s desc = $replace(desc,"III","Ⅲ")
            s desc = $replace(desc," ","")
            ;b:ord=2000306 ;;item
            ;b:item=2
            ;b:item=35
           	Continue:("^"_desc)'=("^"_Type)
           	;b ;:ord=2000306 ;; exe
            Set exe = 0
            For {
                Set exe = $o(^OEORD(ord,"I",item,"X",exe))
                Quit:exe=""
                s stat = $p(^OEORD(ord,"I",item,"X",exe),"^",16)
                ;b ;;; stat
                Continue:stat'=1
                s count = count + 1
            }
		}
	}
	;b:ord=2000306 ;quit
	q Type_"^"_count
}

In [ ]:
/// 删除 医嘱套 恢复
/// d ##class(web.DHCDocMain).Recovery()
ClassMethod Recovery() [ Language = objectscript ]
{
	&sql(DECLARE mycursor CURSOR FOR
		select ObjectDesc ,ObjectReference ,OldValue into :desc , :oref ,:old from SQLUser.DHC_DocDataChangeLog 
		where  
		ClassNameDesc = "医嘱套信息" 
		and OperateType = "D" 
		and DHC_DocDataChangeLog.UpdateDate = current_date-2
		and UpdateUserName = "张成红"
	)
	&sql(OPEN mycursor)
	
	s count = 0
	for {
		&sql(FETCH mycursor)
    	QUIT:SQLCODE'=0
    	if oref [ "User.DHCUserFavItems" {
	    	&sql(
	    		select ObjectReference into :arcos from SQLUser.DHC_DocDataChangeLog 
				where  
				ClassNameDesc = "医嘱套信息" 
				and OperateType = "D" 
				and DHC_DocDataChangeLog.UpdateDate = current_date-2
				and UpdateUserName = "张成红"
				and ObjectDesc = :desc
	    	)
	    	s arcos = $p(arcos,"_",2)
	    	/*
	    	s ^DHCFavItems(4554) = "ARCOS^^9108^10^^^^^^N^2^66872^"

			s $p(^ARCOS(9108),"^",16)=""
	    	*/
	    	s json = {}.%FromJSON(old)
	    	set iter = json.%GetIterator()
			while iter.%GetNext(.key, .value, .type ) {
				;write "key = "_key_", value = "_value_", type = "_type,!
				s user = value."Fav_User_Dr"
				s dept = value."Fav_Dep_Dr"
				s hosp = value."Fav_Hosp_Dr"
				s contral = value."Fav_ContralType"
				s unit = value."Fav_MedUnit_Dr"
				s cele = value."Fav_CelerType"
			}
			s id = $p(oref,"_",2)
	    	s string = $lts(
	    		$lb(
	    			"ARCOS",
	    			user,
	    			arcos,
	    			dept,
	    			"",
	    			contral,
	    			"",
	    			unit,
	    			hosp,
	    			cele,
	    			2,
	    			66872,
	    			""
	    		)
	    		,
	    		"^"
	    	)
	    	;w $d(^DHCFavItems(id)),!
	    	s ^DHCFavItems(id) = string
	    	s $p(^ARCOS(arcos),"^",16)=""
	    	s count = count + 1
	    }
    	
	}
	&sql(CLOSE mycursor)
	w count
	q 0
}


In [ ]:
/// 就诊循环
/// w ##class(web.DHCDocMain).Loop()
ClassMethod AdmLoop() [ Language = objectscript ]
{
	s count = 0
	s id = 0
	;s double = $p(^DHCITMINFO(id),"^",120)
	for {
		s id = $o(^PAADM(id))
		q:id=""
		s rtn = ..GetTransTime(id)
		continue:rtn=""
		s pat = $p(^PAADM(id),"^",1)
		s no = $p(^PAPER(id,"PAT",1),"^",1)
		w no , !
		w rtn , !
		s count = count + 1
		q:count=30
	}
	q count
}



In [ ]:
/// 医嘱 循环
/// w ##class(web.DHCDocMain).OrdLoop()
ClassMethod OrdLoop() [ Language = objectscript ]
{
	s count = 0
	s ord = 0
	;s double = $p(^DHCITMINFO(id),"^",120)
	for {
		s ord = $o(^OEORD(ord))
		q:ord=""
		s item = 0
		for {
			s item = $o(^OEORD(ord,"I",item))
			q:item=""
			s note = $g(^OEORD(ord,"I",item,"DEP",1))
			continue:note'["胸部CT"
			w note,!
			s count = count + 1
			q:count=30
		}
		
	}
	q count
}

In [ ]:
/// 就诊循环
/// w ##class(web.DHCDocMain).Loop()
ClassMethod ArcLoop() [ Language = objectscript ]
{
	s count = 0
	s id = 0
	;s double = $p(^DHCITMINFO(id),"^",120)
	for {
		s id = $o(^ARCIM(id))
		q:id=""
		s rtn = ..GetTransTime(id)
		continue:rtn=""
		s pat = $p(^PAADM(id),"^",1)
		s no = $p(^PAPER(id,"PAT",1),"^",1)
		w no , !
		w rtn , !
		s count = count + 1
		q:count=30
	}
	q count
}

In [ ]:
/// 医嘱子类
s ItemCatRowid=##class(web.DHCDocOrderEntry).GetItemCat(Arcim)


In [ ]:
/*************************************
 *说明：2014-08-15
 *设置添加行数据时 单元格是否可编辑
 *DefaultStyleConfigObj中的属性必须和表格列index相同
 *返回StyleConfigObj
 *如果考虑非强制模式的话，应该是需要在prior的change事件中重算style-tanjishan
 *************************************/
function GetStyleConfigObj(ParamObj)
// 前端样式